dict 类型不但在各种程序里广泛使用，它也是 Python 语言的基石。模块的命名空间、实例的属性和函数的关键字参数中都可以看到字典的身影。跟它有关的内置函数都在 `__builtins__.__dict__`模块中。



正是因为字典至关重要，Python 对它的实现做了高度优化，而散列表则是字典类型性能出众的根本原因。集合（set）的实现其实也依赖于散列表，因此本章也会讲到它。反过
来说，想要进一步理解集合和字典，就得先理解散列表的原理。

## 3.1 范映射类型

In [1]:
tt = (1, 2, (30, 40))

In [2]:
hash(tt)

8027212646858338501

## 3.3 字典推导

字典推导（dictcomp）可以从任何以键值对作为元素的可迭代对象中构建出字典。

In [3]:
DIAL_CODES = [(86, 'China'), (91, 'India'), (1, 'United States'), (62, 'Indonesia'), (55, 'Brazil'), (7, 'Russia')]

In [6]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Russia': 7}

In [8]:
{code: country.upper() for country, code in country_code.items() if code > 60}

{86: 'CHINA', 91: 'INDIA', 62: 'INDONESIA'}

## 3.3 常见的映射方法

映射类型的方法其实很丰富，其中有：dict、defaultdict以及OrderedDict，后面两个是dict的变种，位于collections模块内。

## 3.4 映射的弹性键查询

有时候为了方便起见，就算某个键在映射里不存在，我们也希望在通过这个键读取值的时候能得到一个默认值。有两个途径能帮我们达到这个目的，一个是通过 defaultdict 这个类型而不是普通的 dict，另一个是给自己定义一个 dict 的子类，然后在子类中实现 `__missing__`方
法。



### 3.4.1 defaultdict: 处理找不到的键的一个选择

具体而言，在实例化一个 defaultdict 的时候，需要给构造方法提供一个可调用对象，这个可调用对象会在 `__getitem__` 碰到找不到的键的时候被调用，让 __getitem__ 返回某种默认值。

In [17]:
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list) # 把 list 构造方法作为 default_factory 来创建一个defaultdict。
with open('./chapter3.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            # 如果 index 并没有 word 的记录，那么 default_factory 会被调用，为查询不到的键创造一个值。
            # 这个值在这里是一个空的列表，然后这个空列表被赋值给 index[word]，继而被当作返回值返回，
            # 因此.append(location) 操作总能成功。
            index[word].append(location) 
            
for word in sorted(index, key=str.upper):
    print(word, index[word])

all [(1, 257)]
Always [(3, 145)]
and [(1, 106), (1, 236)]
are [(1, 7)]
be [(1, 182), (1, 202)]
because [(1, 205)]
chance [(1, 244)]
do [(1, 254), (1, 284)]
Dream [(1, 129)]
dream [(1, 152)]
dreams [(1, 99)]
enough [(3, 14), (3, 49), (3, 82), (3, 114)]
feel [(3, 188)]
for [(1, 119)]
from [(1, 89)]
go [(1, 158), (1, 179)]
happiness [(3, 21)]
happy [(3, 138)]
have [(1, 217), (3, 9)]
hope [(3, 121)]
hug [(1, 110)]
human [(3, 108)]
hurts [(3, 201), (3, 223)]
If [(3, 181)]
in [(1, 19), (3, 165)]
it [(3, 198), (3, 211)]
just [(1, 66)]
keep [(3, 99)]
life [(1, 22), (1, 231)]
make [(3, 34), (3, 66), (3, 129)]
May [(3, 1)]
miss [(1, 36)]
moments [(1, 11)]
much [(1, 52)]
one [(1, 227), (1, 240)]
only [(1, 222)]
other [(3, 233)]
others [(3, 168)]
person [(3, 239)]
pick [(1, 79)]
probably [(3, 214)]
put [(3, 152)]
real [(1, 123)]
shoes [(3, 175)]
so [(1, 49)]
someone [(1, 41)]
sorrow [(3, 89)]
strong [(3, 75)]
sweet [(3, 43)]
that [(1, 57), (3, 193)]
the [(1, 261), (3, 229)]
them [(1, 84), (1, 114)

如果在创建 defaultdict 的时候没有指定 default_factory，查询不存在的键会触发 KeyError。

In [23]:
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict() # 把 list 构造方法作为 default_factory 来创建一个defaultdict。
with open('./chapter3.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            # 如果 index 并没有 word 的记录，那么 default_factory 会被调用，为查询不到的键创造一个值。
            # 这个值在这里是一个空的列表，然后这个空列表被赋值给 index[word]，继而被当作返回值返回，
            # 因此.append(location) 操作总能成功。
            index[word].append(location) 
            
for word in sorted(index, key=str.upper):
    print(word, index[word])

KeyError: 'There'

> defaultdict 里的 default_factory 只会在`__getitem__` 里被调用，在其他的方法里完全不会发挥作用。比如，dd 是个 defaultdict，k 是个找不到的键， dd[k] 这个表达式会调用 default_factory 创造某个默认值，而 dd.get(k) 则会返回 None。

所有这一切背后的功臣其实是特殊方法 `__missing__`。它会在defaultdict 遇到找不到的键的时候调用 default_factory，而实际上这个特性是所有映射类型都可以选择去支持的。

### 3.4.2 特殊方法`__missing__`

## 3.5 字典的变种

### collections.OrderedDict

这个类型在添加键的时候会保持顺序，因此键的迭代次序总是一致的。OrderedDict 的 popitem 方法默认删除并返回的是字典里的最后一个元素，但是如果像 my_odict.popitem(last=False) 这样调用它，那么它删除并返回第一个被添加进去的元素。